In [160]:
import tkinter as tk # 視窗介面
from PIL import Image, ImageTk, ImageDraw, ImageFont # 圖片的導入及導出
import numpy as np
from tkinter import filedialog  # 匯入檔案對話框模組

# 目標尺寸
target_width = 499
target_height = 665

# 視窗設定
root = tk.Tk()
root.title("⁂✩❆☃𝄪⊹圖像編輯器⊹𝄪☃❆✩⁂")

# 初始化
original_image = None
image_np = None
img_tk = None
alpha = 1.0  # 對比度
beta = 0.0  # 亮度
input_text = ""  
text_color_depth = 255 
history = []
new_width = 0
new_height = 0

# Panel
panel = tk.Label(root)
panel.grid(row=0, column=0, padx=10, pady=10, sticky=tk.NW)

# 加和減少按鈕的Frame
button_frame = tk.Frame(root)
button_frame.grid(row=0, column=1, padx=10, pady=10, sticky=tk.NW)

def import_image():
    global image_np, original_image, new_width, new_height, add_border_var
    
    # 初始化add_border_var
    add_border_var = tk.BooleanVar()
    add_border_var.set(False)  
    
    image_path = filedialog.askopenfilename()
    if image_path:
        original_image = Image.open(image_path)
        load_image(image_path)


# 載入和顯示圖像
def load_image(image_path):
    global original_image, image_np, new_width, new_height, img_tk
    
    # 根據圖像的寬高比確定縮放後的尺寸!!非常重要!!
    width, height = original_image.size
    if width > height:
        new_size = (target_width, int(target_width * height / width))
    else:
        new_size = (int(target_height * width / height), target_height)
    
    original_image = original_image.resize(new_size) # 縮放
    new_width, new_height = original_image.size
    
    image_np = np.array(original_image) # 轉換為numpy數組
    history.append(image_np.copy())    # 記錄初始圖像 後面有初始化按鈕
    
    update_image()

    
# 更新圖像顯示函數
def update_image():
    global image_np, alpha, beta, input_text, text_color_depth, img_tk, add_border_var
    
    if image_np is None:
        return  # if圖像數據未初始化則返回

    adjusted_image = (image_np.astype(np.float32) * alpha + beta).clip(0, 255).astype(np.uint8)
    
   
    if input_text:     # 在圖像上添加文字
        pil_img = Image.fromarray(adjusted_image)
        draw = ImageDraw.Draw(pil_img)
        font = ImageFont.load_default()  # 默認字體
        
        
        # 計算文字
        text_bbox = draw.textbbox((0, 0), input_text, font=font)   # 大小
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = (new_width - text_width) // 2    # 位置
        text_y = 20
        text_color = (text_color_depth, text_color_depth, text_color_depth)  # 顏色
        
        draw.text((text_x, text_y), input_text, fill=text_color, font=font)  # 在中間位置添加文字
        adjusted_image = np.array(pil_img)
    
    # 邊框效果
    if add_border_var.get():
        border_color = (255, 255, 255)  # 白色邊框
        border_thickness = 10
        adjusted_image = add_border(adjusted_image, border_color, border_thickness)
    
    # 將圖像調整為原始顯示尺寸(以免其他效果計算錯誤)
    adjusted_image_resized = Image.fromarray(adjusted_image).resize((new_width, new_height))
    img_tk = ImageTk.PhotoImage(image=adjusted_image_resized)
    panel.configure(image=img_tk)
    panel.image = img_tk


# 對比度的函數
def adjust_contrast(val):
    global alpha
    alpha = 1 + float(val) / 100.0
    update_image()

# 亮度的函數
def adjust_brightness(val):
    global beta
    beta = float(val)
    update_image()

# 灰階圖函數
def convert_to_grayscale():
    global image_np
    
    gray_weights = np.array([0.2989, 0.587, 0.114]).reshape((1, 1, 3))
    gray_image = np.sum(image_np * gray_weights, axis=2).astype(np.uint8)
    image_np = np.stack([gray_image] * 3, axis=2)
    update_image()
    print("圖像已轉換為灰度")

# 馬賽克函數
def apply_mosaic():
    global image_np
    block_size = 5  # 馬賽克塊大小(影響模糊度)
   
    height, width, _ = image_np.shape   # 確保圖像的寬度和高度都能被block_size整除
    new_height = (height // block_size) * block_size
    new_width = (width // block_size) * block_size
    
    image_np = image_np[:new_height, :new_width]   # 調整圖像尺寸

    for i in range(0, new_height, block_size):      # 應用
        for j in range(0, new_width, block_size):
            block = image_np[i:i+block_size, j:j+block_size]
            center_color = block[block_size // 2, block_size // 2]  # 取中心像素的顏色!!!
            image_np[i:i+block_size, j:j+block_size] = center_color # 將每個區塊填充為中心像素的顏色
    
    update_image()
    print("圖像已打上馬賽克")

#初始化
def reset():
    global image_np, alpha, beta, history, input_text, text_color_depth, add_border_var
    
    
    input_text = ""# 清空文字及入框
    input_text_entry.delete(0, tk.END) 
    
    alpha = 1.0 # 初始對比度
    beta = 0.0 # 初始亮度
    
    text_color_scale.set(255) # 初始滑動條位置
    contrast_scale.set(0)
    brightness_scale.set(0)
    
    image_np = np.array(original_image.resize((new_width, new_height)))  # 去除灰階圖效果
    convert_to_grayscale()
    remove_mosaic() # 去除馬賽克效果
    add_border_var.set(False) # 去除邊框效果
    
    update_image()
    print("圖像已恢復初始狀態")

 # 移除馬賽克效果
def remove_mosaic():
    global image_np
   
    image_np = np.array(original_image.resize((new_width, new_height)))
    update_image()
    print("馬賽克效果已移除")

# 添加邊框效果函數
def add_border(image_array, color, thickness):
    border_image = np.copy(image_array)
    height, width, _ = border_image.shape
    
   
    border_image[:thickness, :, :] = color # 上邊框
   
    border_image[height-thickness:, :, :] = color  # 下邊框
    
    border_image[:, :thickness, :] = color # 左邊框
    
    border_image[:, width-thickness:, :] = color # 右邊框
    
    return border_image

# 添加邊框效果函數
def add_border_effect():
    global add_border_var
    add_border_var.set(True)
    update_image()

# 輸入文字
def input_text_update():
    global input_text
    input_text = input_text_entry.get()
    update_image()
    
# 更新文字顏色深淺度
def set_text_color_depth(val):
    global text_color_depth
    text_color_depth = int(val)
    update_image()
    
# 儲存圖像
def save_image():
    global image_np, alpha, beta, input_text, text_color_depth, add_border_var
    
    # 彈出檔案對話框，讓使用者選擇保存路徑和檔案名稱
    file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                            filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg")])
    
    if file_path:  # 將所有變更內容儲存
        adjusted_image = (image_np.astype(np.float32) * alpha + beta).clip(0, 255).astype(np.uint8)
        
        if input_text:
            pil_img = Image.fromarray(adjusted_image)
            draw = ImageDraw.Draw(pil_img)
            font = ImageFont.load_default() 
            
            text_bbox = draw.textbbox((0, 0), input_text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            text_x = (new_width - text_width) // 2
            text_y = 20
            text_color = (text_color_depth, text_color_depth, text_color_depth)
            
            draw.text((text_x, text_y), input_text, fill=text_color, font=font)
            adjusted_image = np.array(pil_img)
        
        if add_border_var.get():
            border_color = (255, 255, 255)
            border_thickness = 10
            adjusted_image = add_border(adjusted_image, border_color, border_thickness)
        
        pil_image = Image.fromarray(adjusted_image)
        pil_image.save(file_path)
        print(f"保存成功：{file_path}")
        
# 保存圖片按鈕
btn_save = tk.Button(root, text="保存圖片", command=save_image, bg="#FFC0CB")
btn_save.place(relx=0.94, rely=0.70, anchor='n')

# 導入圖片按鈕
btn_import = tk.Button(root, text="導入圖片", command=import_image, bg="#FFC0CB")
btn_import.place(relx=0.83, rely=0.70, anchor='n')

# 添加邊框按鈕
btn_add_border = tk.Button(button_frame, text="添加邊框", command=add_border_effect, bg="#bfefff", width=10, height=1)
btn_add_border.grid(row=4, column=0, padx=5, pady=3, sticky=tk.W)

# 對比度滑動條
contrast_scale = tk.Scale(button_frame, from_=-100, to=100, orient=tk.HORIZONTAL, label="對比度", command=adjust_contrast)
contrast_scale.set(0)  # 初始對比度設置為0，對應alpha=1
contrast_scale.grid(row=1, column=0, padx=2, pady=1, sticky=tk.W)

# 亮度滑動條
brightness_scale = tk.Scale(button_frame, from_=-100, to=100, orient=tk.HORIZONTAL, label="亮度", command=adjust_brightness)
brightness_scale.set(beta)  # 初始亮度設置
brightness_scale.grid(row=0, column=0, padx=5, pady=1, sticky=tk.W)

# 灰度按鈕
btn_grayscale = tk.Button(button_frame, text="灰階圖效果", command=convert_to_grayscale, bg="#bfefff", width=10, height=1)
btn_grayscale.grid(row=2, column=0, padx=5, pady=3, sticky=tk.W)

# 馬賽克按鈕
btn_mosaic = tk.Button(button_frame, text="馬賽克效果", command=apply_mosaic, bg="#bfefff", width=10, height=1)
btn_mosaic.grid(row=3, column=0, padx=5, pady=3, sticky=tk.W)

# 初始化按鈕
btn_reset = tk.Button(root, text="初始化", command=reset)
btn_reset.place(relx=0.95, rely=0.03, anchor='n')

# 輸入文字框
input_text_frame = tk.Frame(root)
input_text_frame.grid(row=1, column=0, columnspan=1, padx=8, pady=10, sticky=tk.W)

input_text_label = tk.Label(input_text_frame, text="輸入要顯示的文字(英文):")
input_text_label.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

input_text_entry = tk.Entry(input_text_frame, width=30)
input_text_entry.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)

# 更新文字按鈕
update_text_button = tk.Button(input_text_frame, text="更新文字", command=input_text_update)
update_text_button.grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)

# 文字顏色深淺調節滑動條
text_color_scale_label = tk.Label(button_frame)
text_color_scale_label.grid(row=5, column=0, padx=5, pady=5, sticky=tk.W)

text_color_scale = tk.Scale(root, from_=0, to=255, orient=tk.HORIZONTAL, label="文字深淺度", command=lambda val: set_text_color_depth(val))
text_color_scale.set(text_color_depth)  # 初始文字顏色深淺度
text_color_scale.place(relx=0.97, rely=0.973, anchor='se')


# 運行!! :)
root.mainloop()


圖像已轉換為灰度
圖像已打上馬賽克
保存成功：C:/Users/candice/Downloads/44444.png
